In [2]:
import itertools
from typing import List

from nltk.lm import Vocabulary
from nltk.lm.models import MLE
from nltk.util import ngrams

In [3]:
N = 3
filepath = "text/peachy/peachy-4304732.txt"

In [6]:
sentences = []
with open(filepath, "r") as f:
    for line in f.readlines()[2:]:
        s = line.strip()
        if len(s) == 0:
            continue
        sentences.append(s)

In [7]:
print("\n".join(sentences))

女性を潤す新たな注目ワードは“アミノ酸”
8月も残すところ10日余り。秋のはじまりと共に気になってくるのがお肌の乾燥ケアだ。潤いとハリのある肌の為にヒアルロン酸とコラーゲンに着目した「ベキュア スキンパーフェクトスフィア」シリーズに、「アミノ酸」に着目したニューアイテムが登場した。
皮膚成分のほとんどはアミノ酸によって占められており、素肌は良質なアミノ酸を常に求めている。今回新シリーズは、アルギニン、保湿作用のあるグルタミン酸、角質を形成するセリン、ハリ感をサポートするプロリンをカプセルに内包し、肌に速攻アプローチ。
特に、ベキュア初となるバームタイプのメイク落としは、毛穴の奥の汚れまですっきり落としつつ、マッサージもできるという1品2役の楽ちんコスメ。マッサージだけだと続けるのは難しいが、毎日行うクレンジングと共に肌に溜まった汚れの排出を促し、すっきりとした輪郭とワントーン明るい肌を手に入れることができる。
また、十分な睡眠時間がとれず、肌本来の再生能力が不足している現代女性の為に睡眠中の肌潤い生理に注目したナイトクリームも登場。眠っている間に潤いとハリ感を与え、朝目覚めた時には艶めく素肌が手に入る。
今年の秋の乾燥ケアにはアミノ酸。忙しい日々で失いがちな潤いを、手軽に補充し、艶肌を手に入れよう。アミノアクトクレンジングバーム、アミノアクトクレンジングフォーム、アミノエフェクティブスリープは8月20日より販売開始。
・ベキュア オンラインショップ


In [8]:
vocabulary = Vocabulary(itertools.chain.from_iterable(sentences))
char_ngram = [ngrams(sentence, N) for sentence in sentences]

In [9]:
list(ngrams(sentences[0], N))[:3]

[('女', '性', 'を'), ('性', 'を', '潤'), ('を', '潤', 'す')]

In [10]:
lm = MLE(order=N, vocabulary=vocabulary)
lm.fit(char_ngram)

In [11]:
context = ("女", "性")
for word in lm.context_counts(lm.vocab.lookup(context)):
    print(f"{word}: {lm.score(word, context)}")

を: 0.5
の: 0.5


In [22]:
text = "アミノアミノクレンジング"
list(ngrams(text, N))[:3]

[('ア', 'ミ', 'ノ'), ('ミ', 'ノ', 'ア'), ('ノ', 'ア', 'ミ')]

In [23]:
test_contexts = list(ngrams(text, N))

In [24]:
scores = [0 for _ in range(len(text))]
i = 0
tp = 0
for context in test_contexts:
    p = lm.score(context[-1], context[:-1])
    print(f"{context}: {p}")
    if p <= tp:
        for j in range(N):
            scores[i+j] -= 1
    i += 1

('ア', 'ミ', 'ノ'): 1.0
('ミ', 'ノ', 'ア'): 0.25
('ノ', 'ア', 'ミ'): 0.0
('ア', 'ミ', 'ノ'): 1.0
('ミ', 'ノ', 'ク'): 0.0
('ノ', 'ク', 'レ'): 0
('ク', 'レ', 'ン'): 1.0
('レ', 'ン', 'ジ'): 1.0
('ン', 'ジ', 'ン'): 1.0
('ジ', 'ン', 'グ'): 1.0


In [25]:
ts = -2
for c, score in zip(list(text), scores):
    if score <= ts:
        print(f"{c}: {score} # error!")
    else:
        print(f"{c}: {score} # ok")

ア: 0 # ok
ミ: 0 # ok
ノ: -1 # ok
ア: -1 # ok
ミ: -2 # error!
ノ: -2 # error!
ク: -2 # error!
レ: -1 # ok
ン: 0 # ok
ジ: 0 # ok
ン: 0 # ok
グ: 0 # ok


In [26]:
def calc_Pf(cm: List[str]) -> float:
    pf = 1.0
    for i in range(2, len(cm)):
        pf *= lm.score(cm[i], (cm[i-2], cm[i-1]))
    return pf

In [27]:
def calc_Pb(cm: List[str]) -> float:
    pb = 1.0
    for i in range(2, len(cm)):
        pb *= lm.score(cm[i-2], (cm[i-1], cm[i]))
    return pb

In [31]:
candidates = dict() # {pos: [word]}
ts = -2
n_candidate_character = 300
n_candidate_word = 5

In [35]:
for i in range(2, len(scores)-2):
    if scores[i] > ts:
        continue

    m1s = []
    context = (text[i-2], text[i-1])
    for m in lm.context_counts(lm.vocab.lookup(context)):
        p = lm.score(m, (context))
        print(f"{''.join(context)}->{m}: {p}")
        if p <= 0:
            continue
        m1s.append([m, p])
    m1s = sorted(m1s, key=lambda x: x[1])[::-1][:n_candidate_character]
    print(i, m1s)

    m2s = []
    for m, p in m1s:
        context = (text[i-1], m)
        for m2 in lm.context_counts(lm.vocab.lookup(context)):
            p2 = lm.score(m2, context)
            print(f"{''.join(context)}->{m2}: {p2}")
            if p2 <= 0:
                continue
            m2s.append([m, m2, p*p2])
    m2s = sorted(m2s, key=lambda x: x[2])[::-1][:n_candidate_character]
    print(i, m2s)

    pfs = []
    pbs = []
    for m, _ in m1s:
        pf = calc_Pf([text[i-2], text[i-1], m, text[i+1], text[i+2]])
        pb = calc_Pb([text[i-2], text[i-1], m, text[i+1], text[i+2]])
        pfs.append([m, pf])
        pbs.append([m, pb])
    pfs = sorted(pfs, key=lambda x: x[1])[::-1][:n_candidate_word]
    pbs = sorted(pbs, key=lambda x: x[1])[::-1][:n_candidate_word]
    can1 = list(set(m for m, _ in pfs+pbs))

    pfs = []
    pbs = []
    for m, m2, _ in m2s:
        pf = calc_Pf([text[i-2], text[i-1], m, m2, text[i+2], text[i+3]])
        pb = calc_Pb([text[i-2], text[i-1], m, m2, text[i+2], text[i+3]])
        pfs.append([m+m2, pf])
        pbs.append([m+m2, pb])
    pfs = sorted(pfs, key=lambda x: x[1])[::-1][:n_candidate_word]
    pbs = sorted(pbs, key=lambda x: x[1])[::-1][:n_candidate_word]
    can2 = list(set(w for w, _ in pfs+pbs))

    candidates[i] = can1 + can2

ノア->ク: 1.0
4 [['ク', 1.0]]
アク->ト: 1.0
4 [['ク', 'ト', 1.0]]
アミ->ノ: 1.0
5 [['ノ', 1.0]]
ミノ->酸: 0.625
ミノ->ア: 0.25
ミノ->エ: 0.125
5 [['ノ', '酸', 0.625], ['ノ', 'ア', 0.25], ['ノ', 'エ', 0.125]]
ミノ->酸: 0.625
ミノ->ア: 0.25
ミノ->エ: 0.125
6 [['酸', 0.625], ['ア', 0.25], ['エ', 0.125]]
ノ酸->”: 0.2
ノ酸->」: 0.2
ノ酸->に: 0.2
ノ酸->を: 0.2
ノ酸->。: 0.2
ノア->ク: 1.0
ノエ->フ: 1.0
6 [['ア', 'ク', 0.25], ['エ', 'フ', 0.125], ['酸', '。', 0.125], ['酸', 'を', 0.125], ['酸', 'に', 0.125], ['酸', '」', 0.125], ['酸', '”', 0.125]]


In [36]:
text

'アミノアミノクレンジング'

In [40]:
"アミノアクトクレンジング" # true

'アミノアクトクレンジング'

In [41]:
candidates

{4: ['ク', 'クト'],
 5: ['ノ', 'ノエ', 'ノ酸', 'ノア'],
 6: ['ア', '酸', 'エ', '酸」', '酸を', '酸。', '酸に', '酸”']}